In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import mafaulda as src

src_file = 'raw_data/air_conditioning/vertiv_digitalis/compressor_top_1.tsv'
kind = 'beaglebone' # beaglebone, boat
NFFT = 512

if kind == 'beaglebone':
    # Fan
    Fs = 4000
    # src_file = 'raw_data/fan_test/fan_speed_3_4000.tsv'
    ts = pd.read_csv(src_file, delimiter='\t', index_col=False, header=None, names=['x', 'y', 'z'])
    # Calculate amplitude in m/s^2 Beaglebone Black ADC and ADXL335 resolution (VIN 1.8V, 12bits)
    for dim in ts.columns:
        ts[dim] = ts[dim] * (1800 / 4096)  # ADC to mV
        ts[dim] = (ts[dim] / 180) * 9.81  # mV to m/s^2 (180 mV/g)
        ts[dim] -= ts[dim].mean()
    ts['t'] = ts.index * (250 / 1000000)

elif kind == 'boat':
    # Boat
    Fs = 200
    #src_file = 'boat/Boat__2.txt'
    ts = pd.read_csv(src_file, delimiter=' ')
    ts['t'] = ts['t'].cumsum()
    for dim in ts.columns:
        ts[dim] -= ts[dim].mean()

ts.set_index('t', inplace=True)
ts

In [ ]:
ts.describe()

In [ ]:
ax = ts[['x', 'y', 'z']].plot(figsize=(20, 10), grid=True, subplots=True)
for i in range(3):
    ax[i].set_xlabel('Time [s]')
    ax[i].set_ylabel(f'Amplitude ({i}) [m/s^2]')
plt.savefig('top_1_waveform.png')

In [ ]:
t = 1
ax = ts[['x', 'y', 'z']].iloc[int(t*Fs):int(t*Fs)+2000].plot(figsize=(20, 10), grid=True, subplots=True)
for i in range(3):
    ax[i].set_xlabel('Time [s]')
    ax[i].set_ylabel(f'Amplitude ({i}) [m/s^2]')
plt.savefig('top_1_waveform_zoom.png')

In [ ]:
x = ts['x']
NFFT = 512
plt.figure(figsize=(20, 4))
Pxx, freqs, t, im = plt.specgram(x, NFFT=NFFT, Fs=Fs, mode='magnitude', scale='dB')
plt.xlabel('Time [s]')
plt.ylabel('Frequency [Hz]')
src.resolution_calc(Fs, NFFT)
plt.savefig('top_1_x_axis_fft_512.png')

In [ ]:
x = ts['y']
plt.figure(figsize=(20, 4))
Pxx, freqs, t, im = plt.specgram(x, NFFT=NFFT, Fs=Fs, mode='magnitude', scale='dB')
plt.xlabel('Time [s]')
plt.ylabel('Frequency [Hz]')
plt.savefig('top_1_y_axis_fft_512.png')

In [ ]:
x = ts['z']
plt.figure(figsize=(20, 4))
Pxx, freqs, t, im = plt.specgram(x, NFFT=NFFT, Fs=Fs, mode='magnitude', scale='dB')
plt.xlabel('Time [s]')
plt.ylabel('Frequency [Hz]')
plt.savefig('top_1_z_axis_fft_512.png')

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20, 4))
t = 1
n = (Fs * t) // NFFT + 1
ax[0].plot(freqs, Pxx.T[n])
ax[0].grid(True)
ax[0].set_yscale('log')
#ax[0].set_xlim(0, 1000)
ax[0].set_xlabel('Frequency [Hz]')
ax[0].set_ylabel('Amplitude [dB]')

ax[1].plot(freqs, Pxx.T[n])
ax[1].grid(True)
#ax[1].set_xlim(0, 1000)
ax[1].set_xlabel('Frequency [Hz]')
ax[1].set_ylabel('Amplitude [m/s^2]')
plt.savefig('top_1_z_axis_fft_512_1sec.png')

In [ ]:
ts[['x', 'y', 'z']].hist(figsize=(20, 5), grid=True, bins=50)

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(20, 4))
ax[0].scatter(ts['x'], ts['y'], s=1)
ax[0].grid(True)
ax[0].set_xlabel('X')
ax[0].set_ylabel('Y')
ax[0].grid(True)

ax[1].scatter(ts['x'], ts['z'], s=1)
ax[1].grid(True)
ax[1].set_xlabel('X')
ax[1].set_ylabel('Z')

ax[2].scatter(ts['y'], ts['z'], s=1)
ax[2].grid(True)
ax[2].set_xlabel('Y')
ax[2].set_ylabel('Z')
#plt.show()
plt.savefig('top_1_orbitals.png')

In [ ]:
smooth = 800
fig, ax = plt.subplots(1, 3, figsize=(20, 4))
ax[0].scatter(
    ts['x'].rolling(window=smooth).mean(),
    ts['y'].rolling(window=smooth).mean(),
    s=1
)
ax[0].grid(True)
ax[0].set_xlabel('X')
ax[0].set_ylabel('Y')
ax[0].grid(True)

ax[1].scatter(
    ts['x'].rolling(window=smooth).mean(),
    ts['z'].rolling(window=smooth).mean(),
    s=1
)
ax[1].grid(True)
ax[1].set_xlabel('X')
ax[1].set_ylabel('Z')
ax[1].grid(True)

ax[2].scatter(
    ts['y'].rolling(window=smooth).mean(),
    ts['z'].rolling(window=smooth).mean(),
    s=1
)
ax[2].grid(True)
ax[2].set_xlabel('Y')
ax[2].set_ylabel('Z')
ax[2].grid(True)

In [ ]:
smooth = 800
fig, ax = plt.subplots(1, 3, figsize=(20, 4))
ax[0].scatter(
    ts['x'].rolling(window=smooth).std(),
    ts['y'].rolling(window=smooth).std(),
    s=1
)
ax[0].grid(True)
ax[0].set_xlabel('X')
ax[0].set_ylabel('Y')
ax[0].grid(True)

ax[1].scatter(
    ts['x'].rolling(window=smooth).std(),
    ts['z'].rolling(window=smooth).std(),
    s=1
)
ax[1].grid(True)
ax[1].set_xlabel('X')
ax[1].set_ylabel('Z')
ax[1].grid(True)

ax[2].scatter(
    ts['y'].rolling(window=smooth).std(),
    ts['z'].rolling(window=smooth).std(),
    s=1
)
ax[2].grid(True)
ax[2].set_xlabel('Y')
ax[2].set_ylabel('Z')
ax[2].grid(True)

In [ ]:
a = ts.iloc[::100, :].interpolate(method='cubic')
fig, ax = plt.subplots(1, 3, figsize=(20, 4))
ax[0].plot(a['x'], a['y'])
ax[0].grid(True)
ax[0].set_xlabel('X')
ax[0].set_ylabel('Y')
ax[0].grid(True)